## **Téléchargement automatique d'images en format IIIF issus de Gallica**

Le format IIIF est utile pour avoir des documents de la meilleure qualité possible. Par exemple, une bonne résolution est nécessaire lors de la création de visualisations comme des cartes interactives.

*Document d'entrée* : fichier .xlsx (type EXCEL) avec la liste des liens gallica ARK renvoyant aux images à télécharger. Les liens sont à indiquer dans le fichier sous leur forme complète : "https://gallica.bnf.fr/ark:/12148/bpt6k6211324q".

*Documents de sortie* : dossier par identifiant ARK dans lequel se trouve toutes les images IIIF (une page = un jpg). Attention : certains documents sur Gallica ne possèdent pas d'images IIIF.

In [ ]:
#@markdown # Connecter le notebook à son compte Google Drive :
#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

# Import des librairies nécessaires
from openpyxl import load_workbook
import pandas as pd
import re
import requests
import shutil
import os
import xml.etree.ElementTree as ET
import lxml
from lxml import etree
from re import match
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
from google.colab import drive
from urllib.error import HTTPError

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/gallica/'#@param {type:"string"}

if not os.path.exists(chemin_vers_le_dossier_de_travail):
      os.makedirs(chemin_vers_le_dossier_de_travail)
os.chdir(chemin_vers_le_dossier_de_travail)

In [ ]:
#@markdown # Récupération des images IIIF :

#@markdown ###Indiquer le chemin vers le fichier Excel contenant les liens Gallica :
chemin_fichier_liens_gallica = '/content/drive/My Drive/gallica/guides_cartes.xlsx'#@param {type:"string"}

#@markdown ###Indiquer le nom de la feuille du fichier Excel où se trouve les liens Gallica (il s'agit des onglets en bas du document) :
nom_feuille = 'Feuille 1' #@param {type:"string"}

#@markdown ###Indiquer la lettre de la colonne où se trouve les liens Gallica :
lettre_colonne = 'A' #@param {type:"string"}

def recup(fichier_xlsx):
  wb = load_workbook(fichier_xlsx)
  ws = wb[nom_feuille]
  column_id = ws[lettre_colonne]
  id_list = [column_id[x].value for x in range(len(column_id))]
  for e in id_list:
    if e == None:
      exit()
    else:
      match = re.findall(r"b[a-z0-9]{4,}", e)
      for id in match:

        # Récupération du nombre de vues du document sur Gallica pour permettre la récupération des images IIIF page par page
        url = "https://gallica.bnf.fr/services/OAIRecord?ark=" + id
        request = urllib.request.Request(url)
        content = urllib.request.urlopen(request)
        parse = BeautifulSoup(content, 'html.parser')
        text = parse.find_all('body')
        vues = parse.find_all('dc:format')
        vues = vues[1]
        nb_pages = int(re.sub(r'[^\d]+', '', str(vues)))
        listpage = range(1, nb_pages)

        # Création d'un dossier par identifiant ARK pour le téléchargement des images IIIF
        os.makedirs(id)
        os.chdir(id)
        
        try:
          for page in listpage:
            filename = "".join([id, "-IIIF", "_", str(page), ".jpg"])
            url2= "".join(["https://gallica.bnf.fr/iiif/ark:/12148/", id, "/f", str(page),"/full/full/0/native.jpg"])
            urllib.request.urlretrieve(url2, filename)
          
            response = requests.get(url2, stream=True)
            with open(filename, 'wb') as out_file:
              shutil.copyfileobj(response.raw, out_file)
              del response
        except HTTPError as tp:
          print("Les images IIIF du document dont l'identifiant ARK est " + id + ", n'ont pas pu être téléchargées.")
          os.chdir(chemin_vers_le_dossier_de_travail)
          os.rmdir(id)

recup(chemin_fichier_liens_gallica)